1. ~~add position variables~~
1. ~~split train/test~~
1. ~~add stepwise into main~~
1. ~~set initial vars~~
1. build corr matrix on variables
1. analize multicollinearity
1. check for linearity between iv and dv
1. add quadratic terms
1. lock in final variables
1. check outliers

In [1]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels
import statsmodels.stats.diagnostic as sms
from statsmodels.formula.api import ols
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.compat import lzip

import math
import seaborn as sns
import numpy as np
import pandas as pd
import csv
import import_ipynb

# import stepregress as sr

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

In [15]:
raw_data = pd.read_csv('data/data_cleaned.csv')
raw_data.reset_index(drop=False, inplace=True)
data_df = raw_data
data_df['Salary_sqrt'] = data_df['Salary'] ** (1/2)
data_df['Salary_log'] = np.log(data_df['Salary'])

train_df, test_df = train_test_split(
     data_df, test_size=0.2, random_state=0, shuffle=True)

In [3]:
train_df['Salary_log'] = np.log(train_df['Salary'])
train_df['RBI_sqrt'] = train_df['RBI'] ** (1/2)
train_df['WAR_ctr'] = train_df['WAR'] + (0 - train_df['WAR'].min())
train_df['WAR_2'] = train_df['WAR_ctr'] ** 2

In [16]:
data_df.head()

,index,IDfg,Name,Age,1B_pos,2B_pos,3B_pos,C_pos,DH_pos,SS_pos,G,PA,AB,R,H,1B,2B,3B,HR,TB,RBI,BB,SO,HBP,IBB,SB,CS,GDP,SH,SF,AVG,OBP,SLG,OPS,ISO,BABIP,wRAA,wRC+,WPA,WAR,Year_FA,seasons_included,Salary,Salary_sqrt,Salary_log
0,0,5677,A.J. Ellis,35,0,0,0,1,0,0,468,1713,1448,143,341,241,63,2,35,513,172,214,311,17,20,2,3,52,19,15,0.235497,0.345391,0.354282,0.699673,0.118785,0.273948,-1.68,92.8,-1.46,2.6,2017,5,2500000.0,1581.138830,14.731801
1,1,746,A.J. Pierzynski,36,0,0,0,1,0,0,664,2587,2424,254,682,479,122,7,74,1040,300,101,278,29,21,5,6,76,12,21,0.281353,0.320878,0.429043,0.749921,0.147690,0.290492,1.20,94.6,-2.58,8.8,2014,5,500000.0,707.106781,13.122363
2,2,746,A.J. Pierzynski,35,0,0,0,1,0,0,664,2628,2455,272,695,489,129,7,70,1048,290,109,273,28,24,5,5,76,15,21,0.283096,0.324611,0.426884,0.751495,0.143788,0.293015,0.00,93.6,-2.85,9.7,2013,5,7500000.0,2738.612788,15.830414
3,3,746,A.J. Pierzynski,37,0,0,0,1,0,0,628,2414,2258,222,616,431,112,7,66,940,288,91,280,33,17,4,6,71,10,22,0.272808,0.312681,0.416298,0.728978,0.143490,0.284385,-0.48,90.4,-4.54,5.3,2015,5,2000000.0,1414.213562,14.508658
4,4,746,A.J. Pierzynski,38,0,0,0,1,0,0,613,2347,2191,217,610,429,107,8,66,931,281,95,278,34,17,1,4,73,4,23,0.278412,0.320339,0.424920,0.745259,0.146508,0.290909,2.42,96.6,-2.46,5.1,2016,5,3000000.0,1732.050808,14.914123


# Model Evaluation

In [ ]:
X_test = test_df[['WAR_ctr', 'WAR_2', 'SLG', 'Year_FA', '2B', '1B_pos', '2B_pos', '3B_pos', 'C_pos',
       'DH_pos', 'SS_pos', 'Age', 'SH']]
X_test = sm.add_constant(X_test)
y_pred = results.predict(X_test)  # predict out of sample
y_true = test_df['Salary_log']
sm.tools.eval_measures.rmse(y_pred, y_true, axis=0)

In [13]:
predictors = ['WAR', 'SLG', 'Year_FA', '2B', '1B_pos', '2B_pos', '3B_pos', 'C_pos',
       'DH_pos', 'SS_pos', 'Age', 'SH']

In [20]:
X = train_df[predictors]
y = train_df[['Salary_log']]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

In [54]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)
kf.get_n_splits(data_df)


5

In [69]:
rmse_list = []
test_idx_list = []
for train_index, test_index in kf.split(test_df):
    test_idx_list.append(test_index)
    X_test = test_df.iloc[test_index][predictors]
    X_test = sm.add_constant(X_test)
    y_pred = results.predict(X_test)
    y_true = test_df.iloc[test_index]['Salary_log']
    rmse = sm.tools.eval_measures.rmse(y_pred, y_true, axis=0)
    rmse_list.append(rmse)

In [70]:
rmse_list

[0.6957865400400174,
 0.706552532488602,
 0.9017282775439202,
 0.7469993435123105,
 0.6201407071213519]

In [77]:
sum(rmse_list) / len(rmse_list)

0.7342414801412404

In [76]:
X_test = test_df[predictors]
X_test = sm.add_constant(X_test)
y_pred = results.predict(X_test)
y_true = test_df['Salary_log']
rmse = sm.tools.eval_measures.rmse(y_pred, y_true, axis=0)
rmse

0.7410537147428496

In [79]:
mae = sm.tools.eval_measures.meanabs(y_pred, y_true, axis=0)
mae

0.5689428273674926